In [6]:
import os
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from flair.data import Sentence
from flair.models import SequenceTagger
import random

/Users/user/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
def get_input():
    count = 0
    with open('adult-corpus.txt', 'w', encoding = 'utf-8') as outfile:
        walk_dir = 'corpora/'
        for root, subdirs, files in os.walk(walk_dir):
            print('--\nroot = ' + root)
            list_file_path = os.path.join(root, 'my-directory-list.txt')
            print('list_file_path = ' + list_file_path)
            for filename in files:
                file_path = os.path.join(root, filename)
                print('\t- file %s (full path: %s)' % (filename, file_path))
                if '.cha' in file_path:
                    with open(file_path, 'r', encoding = 'utf-8') as infile:
                        inlist = infile.read().splitlines()
                        for i in range(5):
                            inlist = (remove_returns(inlist))
                        print (len(inlist))
                        for i in range((len(inlist)) -1):
                            if (inlist[i][0] == '*'
                                    and inlist[i].split()[0] != '*CHI:'
                                    and inlist[i+1].split()[0] == '%mor:'):
                                outfile.write(inlist[i] + '\n')
                                outfile.write(inlist[i+1] + '\n')
                            #outfile.write(item + '\n')
                            #outfile.write('\n')
                    count +=1
    print(count)

def remove_returns(inlist):
    outlist = []
    for i in range(len(inlist)-1):
        if (inlist[i][0] in ['*', '%', '&']
                and inlist[i+1][0] not in ['*', '%', '&']):
            #print('i', inlist[i])
            #print('i+1', inlist[i+1])
            newline = copy.deepcopy(inlist[i]) + copy.deepcopy(inlist[i+1])
            #print('new', newline)
            inlist[i] = newline
            inlist[i+1] = '<<<REMOVE>>>>'
    while '<<<REMOVE>>>>' in inlist:
        inlist.remove('<<<REMOVE>>>>')
    return inlist  

def filter():
    with open('adult-corpus.txt', 'r', encoding = 'utf-8') as infile:
        inlist = infile.read().splitlines()
    print(len(inlist))
    print(inlist[0])
    print(inlist[1])
    with open('adult-corpus-filt-wmor.txt', 'w', encoding = 'utf-8') as outfile:
        for line in inlist:
            #print(line)
            line = filter_line(line)
            line = remove_bracketed_items(line)
            #print(line)
            line = ' '.join(line)
            if ('+/' not in line
                    and '[//' not in line
                    and '+.' not in line
                    and '+/' not in line
                    and '+"/' not in line
                    and '>' not in line
                    and '<' not in line
            ):
                outfile.write(line + '\n')
    print('done')

def remove_bracketed_items(line):
    #line = line.split()
    #print(line[-3:])
    if line[-3:] == ['+', '[+', 'IIN]']:
        line = line[:-3]
    if line[-3:] == ['+', '[+', 'IN]']:
        line = line[:-3]
    if line[-2:] == ['+[+', 'IN]']:
        line = line[:-2]
    for i in range(len(line)):
        #print(line[i][-2:])
        if line[i][-2:] == '[]':
            line[i] = line[i][:-2]
    return(line)

global a_line
def filter_line(line):
    global a_line
    a_line = line
    line = line.split()
    #print(line[-2:])
    if '' in line[-1]:
        line = line[0:-1]
    if line[-1] in ['(1.)', '(2.)', '(3.)', '(4.)', '(5.)', '(6.)', '(7.)']:
        line = line[0:-1]
    if line[-2] in ['(1.)', '(2.)', '(3.)', '(4.)', '(5.)', '(6.)', '(7.)', '(8.)', '(9.)', '(10.)',
                    '(11.)', '(12.)', '(13.)', '(14.)', '(15.)', '(16.)', '(17.)', '(18.)', '(19.)']:
        line = line[0:-2] + [line[-1]]
    if (len(line) > 2
            and line[-2] in ['.', '?', '!']):
        print(line)
        line = line[0:-1]
    for j in range(2):
        if (line[-2] == '[+'
                and line[-1] in ['PI]', 'Pi]', 'R]', 'SR]', 'RES]', 'MLR]', 'IMIT]', 'IN]', 'I]', 'dia]']):
            line = line[:-2]
    if (line[-2] == '.[+'
            and line[-1] in ['PI]', 'Pi]', 'R]', 'SR]', 'RES]', 'MLR]', 'IMIT]', 'IN]', 'I]', 'dia]']):
        line = line[:-2] + ['.']
    for j in range(10):
        for i in range(len(line)):
            if line[i] in ['[?]', '[*]', '[!!]', '[!]']:
                #print(i)
                line = line[0:i] + line[i+1:]
                break
    for j in range(10):
        start, end = 0,0
        for i in range(len(line)):
            if ('[:' in line[i]
                    or '[%' in line[i]
                    or '[x' in line[i]
                    or '[=' in line[i]
                    or '[^' in line[i]
                    or '[*' in line[i]
                    or '[+' in line[i]
                    or ' [:' in line[i]):
                start = i
            if ']' in line[i]:
                end = i
        if (start > 0
                and end > start):
            #print(line)
            line = line[0:start] + line[end+1: len(line)]
            #print('new', line)
    for j in range(5):
        for i in range(len(line)):
            if line[i] == '[/]':
                #print(line)
                line=line[0:i-1] + line[i+1:len(line)]
                #print('new', line)
                break
        for item in line:
            if (item in ['.', ',', '!', '?', '(.)', '„', '‡', '&=laughs', '&=laugh', '&=laughing', '&=cough', '&=whispers',
                    '&=whistle', '&=hums', '&=whistles', '&=roars', '&=squeals', '+"', '+".', '+"/','[/]']
                or '&=' in item):
                line.remove(item)
    for item in line:
        if item == '+"':
            print('yes')
            line.remove(item)
    for i in range(len(line)):
        line[i] = stripstring(line[i])
    return(line)

def stripstring(string):
    outstring = ''
    for item in string:
        if item not in ['(', ')', '?','!', ',', '.']:
            outstring = outstring + item
    return outstring



In [13]:
get_input()
filter()

--
root = corpora/
list_file_path = corpora/my-directory-list.txt
	- file .DS_Store (full path: corpora/.DS_Store)
--
root = corpora/Manchester
list_file_path = corpora/Manchester/my-directory-list.txt
	- file 0metadata.cdc (full path: corpora/Manchester/0metadata.cdc)
--
root = corpora/Manchester/gail
list_file_path = corpora/Manchester/gail/my-directory-list.txt
	- file 020428a.cha (full path: corpora/Manchester/gail/020428a.cha)
2310
	- file 020310a.cha (full path: corpora/Manchester/gail/020310a.cha)
2874
	- file 020310a.cha.rem (full path: corpora/Manchester/gail/020310a.cha.rem)
2874
	- file 020609b.cha (full path: corpora/Manchester/gail/020609b.cha)
1839
	- file 020929a.cha.rem (full path: corpora/Manchester/gail/020929a.cha.rem)
2202
	- file 020616b.cha.rem (full path: corpora/Manchester/gail/020616b.cha.rem)
2081
	- file 020828a.cha (full path: corpora/Manchester/gail/020828a.cha)
2102
	- file 020019b.cha (full path: corpora/Manchester/gail/020019b.cha)
2085
	- file 020407b.c

In [25]:
!grep -v "%mor" adult-corpus-filt-wmor.txt | cut -d":"  -f2 | sed 's/^ //' > adult-corpus-filt1.txt

In [26]:
tagger = SequenceTagger.load('flair/pos-english')
def tag():
    with open('adult-corpus-filt1-flair.txt', 'w', encoding='utf-8') as outfile:
        with open('adult-corpus-filt1.txt', 'r', encoding = 'utf-8') as infile:
            #inlist = infile.read().splitlines()
        #inlist = inlist[0:100]
            count = 0
        #print(len(inlist))
        #print(inlist[411468])
            while True:

                # Get next line from file
                line=infile.readline()

            # if line is empty
            # end of file is reached
                if not line:
                    break
            #print("Line{}: {}".format(count,line.strip()))
            #for line in inlist:
                if len(line) > 0:
                    #    (line[0][0] == '*'
                    #and line[0] != '*CHI:'):
                    count +=1
                    if count % 10000 == 0:
                        print(count)
                    line = line.split()
                    #print(line)
                    #sentence = Sentence(line[1:])
                    sentence = Sentence(line)
                    tagger.predict(sentence)
                    labels = sentence.get_labels('pos')
                    #print(labels)
                    #for label in labels:
                    #    print(label.value)
                    #print(sentences)
                    outfile.write('line: ' + ' '.join(line) + '\n')
                    outfile.write('tags: ')
                    for i in range(len(line)):
                        outfile.write(labels[i].value + ' ')
                    outfile.write('\n')
    infile.close()



2023-08-03 09:35:06,527 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


In [27]:
tag()

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000


In [28]:
def morphemize():
    with open('adult-corpus-filt1-flair.txt', 'r', encoding = 'utf-8') as infile:
        inlist =infile.read().splitlines()
        print(len(inlist))
        print(inlist[0])
        verb_dict = {}
        with open('adult-corpus-filt1-marked-flair+tag.txt', 'w', encoding = 'utf-8') as outfile:
            for i in range(len(inlist)-1):
                if i % 1000000 ==0:
                    print(i)
                if (len(inlist[i]) > 0
                    and len(inlist[i+1]) >0
                    and i % 2 == 0):
                    #and len(inlist[i]) == len(inlist[i+1])):
                    #and inlist[i][0].islower()
                    #and inlist[i+1][0].isupper()):
                        line = inlist[i].split()[1:]
                        labels = inlist[i+1].split()[1:]
                        outline = []
                        #print(line)
                        #print(labels)
                        for i in range(len(labels)):
                            #print(i)
                            while (len(line[i]) > 1
                                and line[i][-1] in ['.', ',', '!', '?', '-']):
                                line[i] = line[i][:-1]
                            if (labels[i] == 'VBZ'
                                and "'" not in line[i]
                                and line[i].lower() not in ['is', 'does', 'has']):
                                    if line[i] in ['goes', 'undergoes', 'echoes', 'focuses', 'passes', 'encompasses', 'possesses', 'expresses']:
                                            outline = outline + [line[i][:-2] + '-3S']
                                    elif (len(line[i]) >3
                                        and line[i][-3:] == 'hes'
                                        and line[i] not in 'breathes'):
                                            outline = outline +[line[i][:-2] + '-3S']
                                    elif (len(line[i]) >4
                                        and line[i][-4:] == 'sses'):
                                            outline = outline +[line[i][:-2] + '-3S']
                                    elif line[i] in ['varies', 'implies', 'applies', 'relies', 'tries', 'occupies', 'studies', 'supplies', 'denies', 'flies', 'embodies', 'replies', 'marries','empties', 'cries', 'carries', 'accompanies', 'complies', 'multiplies', 'dries', 'worries', 'copies', 'buries', 'queries', 'levies']:
                                        outline = outline +[line[i][:-3] + 'y' + '-3S']
                                    elif (len(line[i]) > 3
                                        and(line[i][-4:] == 'fies')):
                                        outline = outline +[line[i][:-3] + 'y' + '-3S']
                                    elif len(line[i]) > 1:
                                        outline = outline + [line[i][:-1] + '-3S']
                                        if  line[i][-2:] == 'es':
                                            #print(line[i])
                                            if line[i] in verb_dict:
                                                verb_dict[line[i]] +=1
                                            else:
                                                verb_dict[line[i]] =1
                            elif (labels[i] in ['VBP', 'VB']
                                and "'" not in line[i]
                                and line[i] not in ['*MOT:']
                                  and line[i].lower() not in ['am', 'are', 'do', 'be', 'have']):
                                    outline = outline + [line[i] + '-BARE']
                            else:
                                outline.append(line[i])
                        #for item in labels:
                        #    print('label', item, item.value)
                        #print(line, outline)
                        outfile.write('line: ' + ' '.join(outline) + '\n')
                        outfile.write('tags: ' + ' '.join(labels) + '\n')
        outbin = []
        #for item in verb_dict:
        #    outbin.append([item, verb_dict[item]])
        #outbin = sorted(outbin, key=lambda x: x[1], reverse=True)
        #for item in outbin:
        #    print(item)

In [29]:
morphemize()

1414976
line: where did we go at the weekend
0
1000000


In [28]:
def get_splits():
    global model_run
    out_dir = 'training-splits/'
    voc = 10
    for max_len in[3,4,5,6,8,10]:
        for model_run in [1,2,3,4,5]:
            print('ml, mr', max_len, model_run)
            train_seqs, test_seqs, token2int = get_cds(max_len, voc)
            print(len(train_seqs), len(test_seqs))
            with open(out_dir + 'adults-lm-max' + str(max_len) + '-split' + str(model_run) + '-train.txt', 'w', encoding = 'utf-8') as outfile:
                for item in train_seqs:
                    #print(item)
                    outfile.write('line: ' + ' '.join(item[0]) + '\n')
                    outfile. write('tags: ' + ' '.join(item[1]) + '\n')
            outfile.close()
            with open(out_dir + 'adults-lm-max' + str(max_len) + '-split' + str(model_run) + '-test.txt', 'w', encoding = 'utf-8') as outfile:
                for item in test_seqs:
                    outfile.write('line: ' + ' '.join(item[0]) + '\n')
                    outfile. write('tags: ' + ' '.join(item[1]) + '\n')
            outfile.close()


In [33]:
def get_cds(max_len = 5, voc =0):
    global word_dict, model_run
    word_dict, token2int = {}, {}
    temp_list, train_list,test_list = [],[],[]
    seq_list_all = []
    templist, line_list,train_bin, test_bin = [], [], [], []
    count1,count2, third_count =0,0,0
    lcount = 0
    infile = 'adult-corpus-filt1-marked-flair+tag.txt'
    #infile = 'subtitles-en-marked-flair+tag.txt'
    #infile = 'subt-sample-wflair.txt'
    indir = ''
    print('input file ', infile)
    print('seq_length =', max_len)
    print('voc =', voc)
    with open(indir + infile, 'r', encoding='utf-8') as infile:
        inlist = infile.read().splitlines()
    ##inlist = inlist[0:1000]
    print(len(inlist))
    count,outline = 0, ''
    word_dict['<eos>'] = 100000
    word_dict['<pad>'] = 100000
    word_dict['-BARE'] = 100000
    word_dict['-3S'] = 100000
    print(word_dict['-BARE'])
    for line in inlist:
        line = line.split()
        for word in line:
            if '-BARE' in word:
                word = word[0:-5]
            if '-3S' in word:
                word = word[0:-3]
            if word not in ['-3S', '-BARE']:
                word = word.lower()
            if word in word_dict:
                word_dict[word] += 1
            else:
                word_dict[word] = 1
    print('total word count', len(word_dict))
    for i in range(len(inlist)):
        lcount +=1
        #if lcount == 10000:
        #    break
        if inlist[i].split()[0] == 'line:':
            line,tags = split_line(inlist[i].split()[1:], inlist[i+1].split()[1:])
            #print(lcount, line,tags)
            for j in range(len(line)):
                index = j + 1
                if j < max_len:
                    temp_list.append([line[0:index], tags[0:index]])
                else:
                    temp_list.append([line[index-max_len:index], tags[index-max_len:index]])
    print('done')
    cnt = 0
    if '<pad>' not in token2int:
        token2int['<pad>'] = cnt
        cnt +=1
    for line in temp_list:
        #print(line)
        for i in range(len(line[0])):
            #if line[0][i] == 'greeting':
            #    print(line[0][i], word_dict[line[0][i]])
            if word_dict[line[0][i]] < voc:
                line[0][i] = '<unk>'

            if line[0][i] not in token2int:
                token2int[line[0][i]] = cnt
                cnt += 1
            # if '<pad>' not in token2int:
            #    token2int['<pad>'] = cnt
        if line[0][-1] in ['-BARE', '-3S']:
            if  random.randint(1,10) < 3:
                #seq_list_all.append(line)
                test_bin.append(line)
                count1 +=1
            else:
                #seq_list_all.append(line)
                train_bin.append(line)
                count2 +=1
            if line[0][-1] == '-3S':
                third_count += 1
        else:
            #seq_list_all.append(line)
            train_bin.append(line)
    #out_dir = 'training-splits/'
    #write_list_to_json(train_bin, out+dir + 'adults-lm-max' + str(max_len) + 'split' + str(model_run) + '-train.txt')
    #write_list_to_json(test_bin, out+dir + 'adults-lm-max' + str(max_len) + 'split' + str(model_run) + '-test.txt')
    #with open('adults-marked1-full-voc10.txt', 'w', encoding = 'utf-8') as outfile:
    #    for line in line_list:
    #        outfile.write(line + '\n')
    print(count1, count2)
    print('third_count', third_count)
    print(train_bin[0])
    print(test_bin[0])
    #train_bin = train_bin[0:10000]
    #train_bin, test_bin =[],[]
    #train_bin, test_bin = read_split(model_run, max_len)
    print(len(train_bin), len(test_bin))
    print(train_bin[0])
    print(test_bin[0])
    return train_bin,test_bin, token2int


def split_line(line, tags):
    outline, outtags = [], []
    for i in range(len(line)):
        if '-BARE' in line[i]:
            outline.append(line[i][0:-5].lower())
            outline.append('-BARE')
            outtags.append(tags[i])
            outtags.append('-BARE')
        elif '-3S' in line[i]:
            outline.append(line[i][0:-3].lower())
            outline.append('-3S')
            outtags.append(tags[i])
            outtags.append('-3S')
        else:
            outline.append(line[i].lower())
            outtags.append(tags[i])
    return outline, outtags


def read_split(run, my_len):
    train_seqs, test_seqs = [],[]
    print('reading split: ', run, 'length = ', my_len)
    in_dir = 'training-splits/'
    print(in_dir + 'adults-lm-max' + str(my_len) + '-split' + str(run) + '-train.txt')
    with open(in_dir + 'adults-lm-max' + str(my_len) + '-split' + str(run) + '-train.txt', 'r', encoding = 'utf-8') as infile:
        inlist = infile.read().splitlines()
    print(inlist[0])
    print(inlist[1])
    print('done')
    #print(len(inlist))
    print(len(inlist))
    for i in range(len(inlist)):
        if inlist[i].split()[0] == 'line:':
            train_seqs.append([inlist[i].split()[1:], inlist[i+1].split()[1:]])
    with open(in_dir + 'adults-lm-max' + str(my_len) + '-split' + str(run) + '-test.txt', 'r', encoding = 'utf-8') as infile:
        inlist = infile.read().splitlines()
    for i in range(len(inlist)):
        if inlist[i].split()[0] == 'line:':
            test_seqs.append([inlist[i].split()[1:], inlist[i + 1].split()[1:]])
    return train_seqs, test_seqs

In [36]:
random.seed(1234)
get_splits()

ml, mr 3 1
input file  adult-corpus-filt1-marked-flair+tag.txt
seq_length = 3
voc = 10
1414976
100000
total word count 12291
done
50331 201441
third_count 13856
[['where'], ['WRB']]
[['we', 'go', '-BARE'], ['PRP', 'VB', '-BARE']]
2988515 50331
[['where'], ['WRB']]
[['we', 'go', '-BARE'], ['PRP', 'VB', '-BARE']]
2988515 50331
ml, mr 3 2
input file  adult-corpus-filt1-marked-flair+tag.txt
seq_length = 3
voc = 10
1414976
100000
total word count 12291
done
50500 201272
third_count 13856
[['where'], ['WRB']]
[['we', 'go', '-BARE'], ['PRP', 'VB', '-BARE']]
2988346 50500
[['where'], ['WRB']]
[['we', 'go', '-BARE'], ['PRP', 'VB', '-BARE']]
2988346 50500
ml, mr 3 3
input file  adult-corpus-filt1-marked-flair+tag.txt
seq_length = 3
voc = 10
1414976
100000
total word count 12291
done
49781 201991
third_count 13856
[['where'], ['WRB']]
[['we', 'go', '-BARE'], ['PRP', 'VB', '-BARE']]
2989065 49781
[['where'], ['WRB']]
[['we', 'go', '-BARE'], ['PRP', 'VB', '-BARE']]
2989065 49781
ml, mr 3 4
input fi

In [8]:
#train_seqs, test_seqs = [],[]
#train_bin, test_bin =[],[]
#word_dict = {}

#train_bin, test_bin =[],[]
#train_seqs, test_seqs = [],[]
#global seq_dict

#voc = 10

#global token2int, int2token, vocab_size
#token2int, int2token = {},{}

#outbin = []
#global net
#global max_len, model_run
#for max_len in [5]:
#    outbin = []
#    #print(x_int[70], y_int[70])
#    for i in range(1):
#        model_run = i+1
#        train_seqs,test_seqs, token2int = get_cds(max_len,voc)


input file  adult-corpus-filt1-marked-flair+tag.txt
seq_length = 5
voc = 10
1414976
100000
total word count 12291
done
50383 201389
third_count 13856
[['where'], ['WRB']]
[['you', 'got', '-BARE'], ['PRP', 'VBP', '-BARE']]
reading split:  1 length =  5
training-splits/adults-lm-max5-split1-train.txt
line: what's
tags: VBZ
done
19358638
9679319 168005
[["what's"], ['VBZ']]
[['and', 'you', 'can', 'sit', '-BARE'], ['CC', 'PRP', 'MD', 'VB', '-BARE']]
